# Model validation - automotive brand dataset
 
Let's now validate the model accross the whole dataset. As evaluation is pretty slow with the brute-force matching,
we will use models specialised to each brand, rather than a general model that can assess every brand. This will naturally
supress the false-positive rate, so this must be kept in mind.

We start with some general initialisation:

In [1]:
import lib.load_data as ld
import lib.model as model
from lib.validation import study_matches
import cv2, os

# Read in the BelgaLogos dataset, filtered by acceptable bounding-boxes and 'OK' flag
metadata = ld.read_metadata()
metadata = ld.filter_by_boundingbox(metadata, 10, 800)
metadata = metadata[metadata.ok == True]

and write a function that generates results from `study_matches` for each brand, summarising them in a dictionary:

In [2]:
def validation_run(metadata, logo_set):
    """Perform a validation run over a list of logos, returning a results dictionary"""
    # Initialise a dictionary for results
    results_list = []
    for logo_name in logo_set:
        # Get the logo image
        logo_filename = os.path.join("data", "logos", logo_name.lower() + '.jpg')
        logo_image = cv2.imread(logo_filename)

        # Initialise model
        SIFT = cv2.xfeatures2d.SIFT_create()
        SIFTMatcher = model.KeypointMatcher(SIFT, cv2.NORM_L2SQR)
        SIFTMatcher.add_template(logo_name, logo_image)

        # Get target dataset
        dataset = metadata[(metadata.brand == logo_name)]

        #Perform study
        results_list.append(study_matches(dataset, SIFTMatcher))
    return results_list

Finally we initialise our brand lists and perform the validation runs

In [3]:
# Initialise a list of car and clothing logo brands
car_logos      = ["Citroen", "Citroen-text", "Ferrari", "Kia", "Mercedes", "Peugeot"]
clothing_logos = ["Adidas", "Airness", "Gucci", "Nike", "Puma", "Puma-text", "Reebok", "Umbro"]

In [ ]:
%%time 
car_results = validation_run(metadata, car_logos) # Run over car logos

In [ ]:
%%time 
clothing_results = validation_run(metadata, clothing_logos) # Run over clothing logos

In [ ]:
from lib.validation import validation_histogram
import importlib
from matplotlib import pyplot as plt
# Plot validation histograms
plt.show(validation_histogram(car_results, car_logos))
plt.show(validation_histogram(clothing_results, car_logos))

In [ ]:
import pandas as pd
import lib.util as ut
car_dataframe = pd.DataFrame(car_results, columns=car_logos)
clothing_dataframe = pd.DataFrame(car_results, columns=car_logos)
ut.multi_table([car_dataframe, clothing_dataframe], ["Car logo summary", "Clothing logo summary"])